In [3]:
from pymongo import MongoClient

# Setup MongoDB client and select database and collection
client = MongoClient('mongodb://localhost:27017/')
db = client['MIMIC-IV']
collection = db['NLP-EXPANDED-prammed-postprocessed_translation']

# Step 1: Identify all non-unique stay_id
pipeline = [
    {"$group": {
        "_id": "$stay_id",
        "count": {"$sum": 1}
    }},
    {"$match": {
        "count": {"$gt": 1}
    }}
]

duplicate_stay_ids = list(collection.aggregate(pipeline))
duplicate_stay_ids = [item["_id"] for item in duplicate_stay_ids if item["_id"] is not None]

# Step 2: Retrieve all documents with these stay_ids
duplicates = {}
if duplicate_stay_ids:
    cursor = collection.find({"stay_id": {"$in": duplicate_stay_ids}}, {'stay_id': 1})
    index = 0
    for doc in cursor:
        stay_id = doc['stay_id']
        if stay_id in duplicates:
            duplicates[stay_id].append(index)
        else:
            duplicates[stay_id] = [index]
        index += 1

# Now `duplicates` contains the stay_id as key and list of indices as values
for stay_id, indices in duplicates.items():
    print(f"stay_id: {stay_id}; indices: {indices}")

stay_id: 30394981; indices: [0, 27]
stay_id: 38848718; indices: [1, 28]
stay_id: 38320237; indices: [2, 29]
stay_id: 35420907; indices: [3, 30]
stay_id: 31198145; indices: [4, 31]
stay_id: 37238447; indices: [5, 32]
stay_id: 30238398; indices: [6, 33]
stay_id: 30942855; indices: [7, 34]
stay_id: 34278605; indices: [8, 35]
stay_id: 32126227; indices: [9, 36]
stay_id: 31151696; indices: [10, 37]
stay_id: 39984868; indices: [11, 38]
stay_id: 36292453; indices: [12, 39]
stay_id: 37351332; indices: [13, 40]
stay_id: 36596503; indices: [14, 41]
stay_id: 32292711; indices: [15, 42]
stay_id: 30114157; indices: [16, 43]
stay_id: 34401029; indices: [17, 44]
stay_id: 35382782; indices: [18, 45]
stay_id: 39476258; indices: [19, 46]
stay_id: 33315656; indices: [20, 47]
stay_id: 34817775; indices: [21, 48]
stay_id: 35008915; indices: [22, 49]
stay_id: 39699150; indices: [23, 50]
stay_id: 38086634; indices: [24, 51]
stay_id: 35678991; indices: [25, 52]
stay_id: 37961886; indices: [26, 53]
stay_id: 31

In [8]:
from pymongo import MongoClient

# Setup MongoDB client and select database
client = MongoClient('mongodb://localhost:27017/')
db = client['MIMIC-IV']

# Specify the source collection
source_collection = db['NLP-EXPANDED-prammed-postprocessed_translation']

# Specify the target collection with the postfix
target_collection_name = 'NLP-EXPANDED-prammed-postprocessed_translation-copysave-predoublettesdeletion'
target_collection = db[target_collection_name]

# Check if the target collection already exists to avoid overwriting data
existing_collections = db.list_collection_names()
if target_collection_name in existing_collections:
    print(f"Collection '{target_collection_name}' already exists.")
else:
    # Copy all documents from the source to the target collection
    documents = source_collection.find()
    target_collection.insert_many(documents)  # Use insert_many to copy documents
    print(f"All documents copied to {target_collection_name}.")

All documents copied to NLP-EXPANDED-prammed-postprocessed_translation-copysave-predoublettesdeletion.


In [10]:
from pymongo import MongoClient

def list_older_duplicate_ids(collection_name):
    client = MongoClient('mongodb://localhost:27017/')
    db = client['MIMIC-IV']
    collection = db[collection_name]

    # Aggregate documents to find duplicates based on 'stay_id'
    pipeline = [
        {"$group": {
            "_id": "$stay_id",
            "all_ids": {"$push": "$_id"},
            "count": {"$sum": 1}
        }},
        {"$match": {
            "count": {"$gt": 1}  # Ensure there are duplicates
        }}
    ]

    duplicates = list(collection.aggregate(pipeline))

    # List of all older duplicate _ids
    older_duplicate_ids = []

    # Process each group of duplicates
    for duplicate in duplicates:
        stay_id = duplicate["_id"]
        ids = duplicate["all_ids"]
        ids.sort()  # Sorting the ids (ObjectId) to find older entries
        
        # Add all but the most recent document's _id to the list
        older_ids = ids[:-1]  # Keep the last one, list others
        older_duplicate_ids.extend(older_ids)
    
    return older_duplicate_ids

# Example Usage
older_duplicate_ids = list_older_duplicate_ids('NLP-EXPANDED-prammed-postprocessed_translation')
older_duplicate_ids

[ObjectId('66ac856a070f0b6e27cee826'),
 ObjectId('66ac890c070f0b6e27cee837'),
 ObjectId('66ac87e1070f0b6e27cee833'),
 ObjectId('66ac9c47070f0b6e27cee88a'),
 ObjectId('66ac84db070f0b6e27cee823'),
 ObjectId('66ac9cef070f0b6e27cee88d'),
 ObjectId('66ac84b0070f0b6e27cee822'),
 ObjectId('66ac85b7070f0b6e27cee828'),
 ObjectId('66ac9b73070f0b6e27cee886'),
 ObjectId('66ac8541070f0b6e27cee825'),
 ObjectId('66ac850d070f0b6e27cee824'),
 ObjectId('66ac9c74070f0b6e27cee88b'),
 ObjectId('66ac866d070f0b6e27cee82b'),
 ObjectId('66ac9bbb070f0b6e27cee887'),
 ObjectId('66ac9a40070f0b6e27cee882'),
 ObjectId('66ac8591070f0b6e27cee827'),
 ObjectId('66ac8868070f0b6e27cee834'),
 ObjectId('66ac863b070f0b6e27cee82a'),
 ObjectId('66ac83ec070f0b6e27cee81f'),
 ObjectId('66ac841b070f0b6e27cee820'),
 ObjectId('66ac9cb6070f0b6e27cee88c'),
 ObjectId('66ac86c3070f0b6e27cee82d'),
 ObjectId('66ac8796070f0b6e27cee831'),
 ObjectId('66ac888f070f0b6e27cee835'),
 ObjectId('66ac9bed070f0b6e27cee888'),
 ObjectId('66ac83c1070f0b

In [11]:
import pandas as pd
from pymongo import MongoClient

def create_df_of_older_duplicates(collection_name):
    client = MongoClient('mongodb://localhost:27017/')
    db = client['MIMIC-IV']
    collection = db[collection_name]

    # Aggregate documents to find duplicates based on 'stay_id'
    pipeline = [
        {"$group": {
            "_id": "$stay_id",
            "all_ids": {"$push": "$_id"},
            "count": {"$sum": 1}
        }},
        {"$match": {
            "count": {"$gt": 1}  # Ensure there are duplicates
        }}
    ]

    duplicates = list(collection.aggregate(pipeline))

    # Prepare data for DataFrame
    data = []
    for duplicate in duplicates:
        stay_id = duplicate["_id"]
        ids = duplicate["all_ids"]
        ids.sort()  # Sorting the ids (ObjectId) to find older entries

        # Keep all but the most recent document's _id
        older_ids = ids[:-1]
        for oid in older_ids:
            data.append({"stay_id": stay_id, "_id": oid})

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Example Usage
df_older_duplicates = create_df_of_older_duplicates('NLP-EXPANDED-prammed-postprocessed_translation')
df_older_duplicates

,stay_id,_id
0,32126227,66ac856a070f0b6e27cee826
1,36292453,66ac85e5070f0b6e27cee829
2,35382782,66ac873b070f0b6e27cee82f
3,34401029,66ac8702070f0b6e27cee82e
4,35678991,66ac88cc070f0b6e27cee836
5,31198145,66ac844a070f0b6e27cee821
6,36208077,66ac9b38070f0b6e27cee885
7,30394981,66ac8391070f0b6e27cee81d
8,34392346,66ac9b0a070f0b6e27cee884
9,34817775,66ac87b9070f0b6e27cee832


In [15]:
import pandas as pd

def remove_rows_by_id(df, ids_to_remove):
    """
    Removes rows from a DataFrame based on a list of _id values.

    :param df: The original DataFrame.
    :param ids_to_remove: A list of _id values to remove from the DataFrame.
    :return: A new DataFrame with the specified rows removed.
    """
    # Convert the list of ids to remove into a set for faster lookup
    ids_to_remove_set = set(ids_to_remove)
    
    # Filter the DataFrame to exclude rows with _id values in the set
    filtered_df = df[~df['_id'].isin(ids_to_remove_set)]
    return filtered_df


'''
# Example DataFrame creation (assuming you have it loaded similarly)
data = {
    'stay_id': [
        32126227, 36292453, 35382782, 34401029, 35678991, 31198145,
        36208077, 30394981, 34392346, 34817775, 35718136, 39476258,
        32292711, 36776235, 32234793, 35618787, 35315273, 38848718,
        35421113, 33315656, 38086634, 30114157, 33206147, 38320237,
        35420907, 37351332, 39699150, 31346589, 31151696, 36591813,
        30942855, 36596503, 30946765, 34278605, 39984868, 39224356,
        37238447, 35008915, 39197730, 30238398, 37961886
    ],
    '_id': [
        '66ac856a070f0b6e27cee826', '66ac85e5070f0b6e27cee829', '66ac873b070f0b6e27cee82f',
        '66ac8702070f0b6e27cee82e', '66ac88cc070f0b6e27cee836', '66ac844a070f0b6e27cee821',
        '66ac9b38070f0b6e27cee885', '66ac8391070f0b6e27cee81d', '66ac9b0a070f0b6e27cee884',
        '66ac87b9070f0b6e27cee832', '66ac9cef070f0b6e27cee88d', '66ac8766070f0b6e27cee830',
        '66ac869b070f0b6e27cee82c', '66ac9c18070f0b6e27cee889', '66ac9d1b070f0b6e27cee88e',
        '66ac9a97070f0b6e27cee883', '66ac9d45070f0b6e27cee88f', '66ac83c1070f0b6e27cee81e',
        '66ac9bed070f0b6e27cee888', '66ac8796070f0b6e27cee831', '66ac888f070f0b6e27cee835',
        '66ac86c3070f0b6e27cee82d', '66ac9cb6070f0b6e27cee88c', '66ac83ec070f0b6e27cee81f',
        '66ac841b070f0b6e27cee820', '66ac863b070f0b6e27cee82a', '66ac8868070f0b6e27cee834',
        '66ac9a40070f0b6e27cee882', '66ac8591070f0b6e27cee827', '66ac9c74070f0b6e27cee88b',
        '66ac850d070f0b6e27cee824', '66ac866d070f0b6e27cee82b', '66ac9bbb070f0b6e27cee887',
        '66ac8541070f0b6e27cee825', '66ac85b7070f0b6e27cee828', '66ac9b73070f0b6e27cee886',
        '66ac84b0070f0b6e27cee822', '66ac87e1070f0b6e27cee833', '66ac9c47070f0b6e27cee88a',
        '66ac84db070f0b6e27cee823', '66ac890c070f0b6e27cee837'
    ]
}

df = pd.DataFrame(data)
'''
# IDs to remove (from your list)
ids_to_remove = [
    '66ac856a070f0b6e27cee826', '66ac85e5070f0b6e27cee829', '66ac873b070f0b6e27cee82f',
    '66ac8702070f0b6e27cee82e', '66ac88cc070f0b6e27cee836', '66ac844a070f0b6e27cee821',
    '66ac9b38070f0b6e27cee885', '66ac8391070f0b6e27cee81d', '66ac9b0a070f0b6e27cee884',
    '66ac87b9070f0b6e27cee832', '66ac9cef070f0b6e27cee88d', '66ac8766070f0b6e27cee830',
    '66ac869b070f0b6e27cee82c', '66ac9c18070f0b6e27cee889', '66ac9d1b070f0b6e27cee88e',
    '66ac9a97070f0b6e27cee883', '66ac9d45070f0b6e27cee88f', '66ac83c1070f0b6e27cee81e',
    '66ac9bed070f0b6e27cee888', '66ac8796070f0b6e27cee831', '66ac888f070f0b6e27cee835',
    '66ac86c3070f0b6e27cee82d', '66ac9cb6070f0b6e27cee88c', '66ac83ec070f0b6e27cee81f',
    '66ac841b070f0b6e27cee820', '66ac863b070f0b6e27cee82a', '66ac8868070f0b6e27cee834',
    '66ac9a40070f0b6e27cee882', '66ac8591070f0b6e27cee827', '66ac9c74070f0b6e27cee88b',
    '66ac850d070f0b6e27cee824', '66ac866d070f0b6e27cee82b', '66ac9bbb070f0b6e27cee887',
    '66ac8541070f0b6e27cee825', '66ac85b7070f0b6e27cee828', '66ac9b73070f0b6e27cee886',
    '66ac84b0070f0b6e27cee822', '66ac87e1070f0b6e27cee833', '66ac9c47070f0b6e27cee88a',
    '66ac84db070f0b6e27cee823', '66ac890c070f0b6e27cee837'
]

# Remove the rows
df_filtered = remove_rows_by_id(df, ids_to_remove)
print(df_filtered)

Empty DataFrame
Columns: [stay_id, _id]
Index: []
